# MOWCATL algorithm

### Data

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
store_data = pd.read_csv(r"D:\Download\COVID19\association_rule.csv")  #Importing Dataset from system

store_data.head()   # to check the header

store_data = pd.read_csv(r"D:\Download\COVID19\association_rule.csv", header=None)  #keeping header as None

num_records = len(store_data)
print(num_records)

22


In [3]:
cleaned_data = store_data.iloc[:, :18]
cleaned_data

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,NewCasesMonthlyChange,NewDeathsMonthlyChange,School_closing,Workplace_closing,Cancel_public_events,Gatherings_restrictions,Close_public_transport,stay_home_requirements,movement_restrictions,International_travel_controls,Income_support,Debt_contract_relie,Public_information_campaigns,Testing_policy,Contact_tracing,Gov_stringency_Index,Government_Response,Containment_Health
1,no_cases,no_deaths,School_normal,Workplace_normal,Public_events_normal,No_gatherings_restrictions,transport_normal,stay_home_not_require,no_movement_restrictions,no_travel_control,no_income_support,no_relie,no_campaign,no_testing_policy,no_contact_tracing,gov_stringency_none,Government_Response_none,ContainmentHealth_none
2,no_cases,no_deaths,School_normal,Workplace_normal,Public_events_normal,No_gatherings_restrictions,transport_normal,stay_home_not_require,no_movement_restrictions,no_travel_control,no_income_support,no_relie,no_campaign,no_testing_policy,no_contact_tracing,gov_stringency_none,Government_Response_none,ContainmentHealth_none
3,no_cases,no_deaths,School_normal,Workplace_normal,Public_events_normal,No_gatherings_restrictions,transport_normal,stay_home_not_require,no_movement_restrictions,no_travel_control,no_income_support,no_relie,no_campaign,no_testing_policy,no_contact_tracing,gov_stringency_none,Government_Response_none,ContainmentHealth_none
4,no_cases,no_deaths,School_normal,Workplace_normal,Public_events_normal,No_gatherings_restrictions,transport_normal,stay_home_not_require,no_movement_restrictions,no_travel_control,no_income_support,no_relie,no_campaign,no_testing_policy,no_contact_tracing,gov_stringency_none,Government_Response_none,ContainmentHealth_none
5,no_cases,no_deaths,School_normal,Workplace_normal,Public_events_normal,No_gatherings_restrictions,transport_normal,stay_home_not_require,no_movement_restrictions,no_travel_control,no_income_support,no_relie,no_campaign,no_testing_policy,no_contact_tracing,gov_stringency_none,Government_Response_none,ContainmentHealth_none
6,no_cases,no_deaths,School_normal,Workplace_normal,Public_events_normal,No_gatherings_restrictions,transport_normal,stay_home_not_require,no_movement_restrictions,no_travel_control,no_income_support,no_relie,no_campaign,no_testing_policy,no_contact_tracing,gov_stringency_none,Government_Response_none,ContainmentHealth_none
7,no_cases,no_deaths,School_normal,Workplace_normal,Public_events_normal,No_gatherings_restrictions,transport_normal,stay_home_not_require,no_movement_restrictions,no_travel_control,no_income_support,no_relie,no_campaign,no_testing_policy,no_contact_tracing,gov_stringency_none,Government_Response_none,ContainmentHealth_none
8,no_cases,no_deaths,School_normal,Workplace_normal,Public_events_normal,No_gatherings_restrictions,transport_normal,stay_home_not_require,no_movement_restrictions,no_travel_control,no_income_support,no_relie,no_campaign,no_testing_policy,no_contact_tracing,gov_stringency_none,Government_Response_none,ContainmentHealth_none
9,no_cases,no_deaths,School_normal,Workplace_normal,Public_events_normal,No_gatherings_restrictions,transport_normal,stay_home_not_require,no_movement_restrictions,no_travel_control,no_income_support,no_relie,no_campaign,no_testing_policy,no_contact_tracing,gov_stringency_none,Government_Response_none,ContainmentHealth_none


In [4]:
target_data = store_data[[18, 19, 20, 21]]
target_data.columns = [0, 1, 2, 3]
target_data

,0,1,2,3
0,Employed_total,Full_time,Part_time,Unemployment_rate
1,total_employed_medium,Full_time_employed_medium,Part_time_employed_low,Unemployment_rate_medium
2,total_employed_high,Full_time_employed_high,Part_time_employed_medium,Unemployment_rate_low
3,total_employed_medium,Full_time_employed_medium,Part_time_employed_high,Unemployment_rate_low
4,total_employed_high,Full_time_employed_medium,Part_time_employed_high,Unemployment_rate_low
5,total_employed_high,Full_time_employed_medium,Part_time_employed_very_high,Unemployment_rate_low
6,total_employed_high,Full_time_employed_medium,Part_time_employed_very_high,Unemployment_rate_low
7,total_employed_high,Full_time_employed_high,Part_time_employed_high,Unemployment_rate_low
8,total_employed_high,Full_time_employed_medium,Part_time_employed_very_high,Unemployment_rate_low
9,total_employed_high,Full_time_employed_high,Part_time_employed_very_high,Unemployment_rate_low


### 1. generate all initial episodes in k = 1

In [5]:
import itertools

def init_all_epsiode(data, min_supp):
    
    all_antecedent_list = {}
    for i in range(len(data.columns)):
        print("this is ", i, "col")
        antecedent_list = init_epsiode(data[[i]], min_supp)
        temp = [antecedent_list]
        all_antecedent_list.update({i : temp})
    
    return all_antecedent_list


def init_epsiode(data, min_supp):
    
    antecedent_list = []
    
    GC_list = data.values
    GC_list = list(itertools.chain.from_iterable(GC_list))

    _t = list(np.unique(GC_list))   
    
    for item in _t:
        #print(item)
        item = [item]
        count = init_count(item, GC_list)
        if count > min_supp:
            antecedent_list.append(item)
            print(item,count, "is support" )
       
    return antecedent_list


def init_count(item_set, data):
    # we need to go over the all data, find how many times the item set is appear.
    #print(data)
    count = 0
    for i in range(len(item_set)):
        for j in range(len(data)):

            #print(set(item_set), set([data[j]]))
            #print(set(item_set).issubset(set(data[j])))

            if set(item_set).issubset(set([data[j]])):
                count += 1
        
    return count

### 2. build all combination for single space

In [6]:
from itertools import combinations, chain

def powerset(s):
    return list(chain.from_iterable(combinations(s,r) for r in range(1,len(s)+1)))

def get_all_epsiodes(data, L1_item_sets, min_supp, window):
    
    item_sets = {}
    item_sets_occurences = {}
    item_sets_count = {}
    
    # get data
    # know how many space
    # do one space each time, along with its item_set
    length = len(data.columns)
    for i in range(length):
        print("begin data space:", i)
        #print(data[[i]])
        #print(L1_item_sets[i])
        item_set, item_occurence, item_count = check_epsiode(data[[i]], L1_item_sets[i], min_supp, window)
        # update dict
        item_sets.update({i: item_set})
        item_sets_occurences.update({i : item_occurence})
        item_sets_count.update({i : item_count})
    
    return item_sets, item_sets_occurences, item_sets_count


def check_epsiode(data, L1_item_set, min_supp, window):
    
    item_set = []
    item_occurence = []
    item_count = []
    # first generate all combination
    # pop the combination one by one
    # count if it meet the min_supp
    s = powerset(L1_item_set[0])
    s.pop()
    
    for z in s:
        #print(z)
        count, min_occurences = count_epsiode(data, z, window)
        if count > min_supp:
            print("item set candidate:", z)
            print("occur in :", min_occurences)
            print("count :", count)
            # add this item to list
            # add its occurences to list
            # add its count to list
            item_set.append(z)
            item_occurence.append(min_occurences)
            item_count.append(count)
   
    return item_set, item_occurence, item_count

def count_epsiode(data, item, window):
    
    count = 0
    minimal_occurences = []
    
    if len(item) <= window :     
    
        # count epsiode
        # also record its minimal occurences
        # when the item not larger than window, process it
        for i in range(len(data)- window):
            find, begin, end = sequence_check(data, i, item, window)
            if find:
                occur = "{}-{}".format(begin, end)
                if minimal_occurences.count(occur) == 0 :
                    # new position found for this epsiode
                    minimal_occurences.append(occur)
                    count += 1
                
    return count, minimal_occurences

In [7]:
def sequence_check(data, start_index, item, window):
    # we will try to find every item in epsiode inside the search window
    item_mark = len(item)
    item_flag = 0
    begin_position = 0
    end_position = 0
    
    #print("item to check ", item, item_mark)
    locol_data = data.copy()
    locol_data.columns = [0]
    
    for i in range(start_index, start_index + window):
        #print(i)
        #print(locol_data.iloc[i][0], item[item_flag][0])
        if locol_data.iloc[i][0] == item[item_flag][0]:
            if begin_position == 0:
                begin_position = i
                end_position = i
            else:
                end_position = i
            item_flag += 1
            
            if item_flag == item_mark:
                # this means we find all item in epsiode
                return True,begin_position, end_position
    
    return False, 0, 0

### 3. Create initial dataframe

In [8]:
def create_init_dataframe(antecedent_item_sets, antecedent_item_sets_occurences, antecedent_item_sets_count,
                         consequent_item_sets, consequent_item_sets_occurences, consequent_item_sets_count):
    
    df1 = create_init_ante_dataframe(antecedent_item_sets, antecedent_item_sets_occurences, antecedent_item_sets_count)
    df2 = create_init_conse_dataframe(consequent_item_sets, consequent_item_sets_occurences, consequent_item_sets_count)
    df_final = pd.concat([df1, df2])
    
    return df_final

def create_init_ante_dataframe(antecedent_item_sets, antecedent_item_sets_occurences, antecedent_item_sets_count):
    
    first_col = "episode/rule"
    second_col = "consequent"
    third_col = "Minimal occurrences"
    forth_col = "Support"
    fifth_col = "confidence"
    
    first_list = []
    second_list = []
    third_list = []
    forth_list = []
    fifth_list = []
    
    df = pd.DataFrame(columns = [first_col,second_col,third_col,forth_col, fifth_col])
    
    if len(antecedent_item_sets) > 0:

        for k,v in  antecedent_item_sets.items():
            
            #print(k, v)
            #print(len(v))
            
            for i in range(len(v)):
                first_list.append(v[i])
                second_list.append("#")
                third_list.append(antecedent_item_sets_occurences[k][i])
                forth_list.append(antecedent_item_sets_count[k][i])
                fifth_list.append("#")
    
        data = {first_col:  first_list,
                second_col: second_list,
                third_col: third_list,
                forth_col: forth_list,
                fifth_col: fifth_list
                }

        df = pd.DataFrame (data, columns = [first_col,second_col,third_col,forth_col,fifth_col])
        
    return df


def create_init_conse_dataframe(consequent_item_sets, consequent_item_sets_occurences, consequent_item_sets_count):
    first_col = "episode/rule"
    second_col = "consequent"
    third_col = "Minimal occurrences"
    forth_col = "Support"
    fifth_col = "confidence"
    
    first_list = []
    second_list = []
    third_list = []
    forth_list = []
    fifth_list = []
    
    df = pd.DataFrame(columns = [first_col,second_col,third_col,forth_col,fifth_col])
    
    if len(consequent_item_sets) > 0:

        for k,v in  consequent_item_sets.items():
            
            #print(k, v)
            #print(len(v))
            
            for i in range(len(v)):
                first_list.append("#")
                second_list.append(v[i])
                third_list.append(consequent_item_sets_occurences[k][i])
                forth_list.append(consequent_item_sets_count[k][i])
                fifth_list.append("#")
    
        data = {first_col:  first_list,
                second_col: second_list,
                third_col: third_list,
                forth_col: forth_list,
                fifth_col: fifth_list
                }

        df = pd.DataFrame (data, columns = [first_col,second_col,third_col,forth_col,fifth_col])
        
    return df

### 4. get epsiode combination in all space 

In [9]:
def get_space_epsiode_combination(item_sets, space_order):
    
    all_space_epsiode_list = []
    
    # first we need to add all signle epsiode into storage
    # k = 1
    for k,v in item_sets.items():
        for i in range(len(v)):
            temp = {k : v[i]}
            all_space_epsiode_list.append(temp)
    
    # second we need to deal with k = 2 combination
    
    new_candidate = []
    the_initial_space = list(item_sets)
    for i in range(len(space_order) -1):
        #print(the_initial_space[i])
        for j in range(i+1, len(space_order)):
        
            print(i,j)
            
            first_group_candidate = list(item_sets[the_initial_space[i]])
            print(first_group_candidate)
            second_group_candidate = list(item_sets[the_initial_space[j]])
            print(second_group_candidate)
            for x in range(len(first_group_candidate)):
                for y in range(len(second_group_candidate)):
                    temp = merge_candidate(the_initial_space[i],first_group_candidate[x], 
                                                        the_initial_space[j], second_group_candidate[y], space_order)
                    all_space_epsiode_list.append(temp)
                    new_candidate.append(temp)
                    
    
    # last we want to deal with k > 2 combination


    while len(new_candidate) > 2:
        print("=============")
        print("this round has candidate:", len(new_candidate))
        print("=============")
        Ln = []
        # we start doing combination
        # always resign the new_candidate
        for i in range(0,len(new_candidate)-1):
            for j in range(i+1,len(new_candidate)):
                #print(list(L2_item_set[i]), list(L2_item_set[j]))
                #print(L2_item_set[i], L2_item_set[j])

                item_set = merge_candidate(list(new_candidate[i]),new_candidate[i], list(new_candidate[j]), new_candidate[j],space_order)

                if len(item_set) > 0:
                    print(item_set)
                    # we need to check if duplicate
                    if check_duplicate(Ln,item_set):
                        print("duplicate")
                    else:
                        Ln.append(item_set)
                        all_space_epsiode_list.append(item_set)
        new_candidate = Ln
        
        
    return all_space_epsiode_list

def merge_item_list(item1, item2, order):
    length = len(item1)
    count = 0
    same_space = []
    temp = item1.copy()
    temp2 = item2.copy()

    for i in order:
        if (i in item1) & (i in item2):
            if item1[i] == item2[i]:
                count += 1
                #print(i, "is the same space")
                same_space.append(i)
    
    if (length - count) == 1:
        # we will merge item here with the support of order
        print(item1)
        print(item2)
        print("item allow to merge")
        for j in range(len(same_space)):
            del temp2[same_space[j]]
        
        temp.update(temp2.items())
        return temp
        
    return {}

def merge_candidate(space1, item1, space2, item2, order):
    
    temp_dict = {}
    
    if isinstance(space1, list): 
        #print("your object is a list, proceeding for L > 2 ") 
        # this means the item we received will be multiple, we will have to
        # apply special check for them
        
        #print(space1)
        #print(len(space1))
        #print(space2)
        #print(len(space2))
        
        if len(space1) == len(space2):
            mark = len(space1)
            space_same_count = 0  
            #print("same length")
        
            # test if the two space are only different in 1 space
            for i in range(len(space1)):
                for j in range(len(space1)):
                    if space1[i] == space2[j]:
                        space_same_count += 1
                    
            # if the difference in space is 1, we are going to check if the same space has same item
            # we need to trace the order of space
            
            if (mark-space_same_count) == 1:
                #print("space allow to merge")
                temp_dict = merge_item_list(item1, item2, order)
                return temp_dict
            #else:
                #print("not allow to merge")
        #print("====end====")
        
    else: 
        #print("your object is not a list, this is the initial L2") 
    
        if space1 != space2:
            temp_dict.update({space1 : item1})
            temp_dict.update({space2 : item2})

        print("direct merge:", temp_dict)
    return temp_dict 


def check_duplicate(all_rule_set, item_set):
    result = False
    temp = {}
    
    for i in range(len(all_rule_set)):
        count = 0
        temp = all_rule_set[i]
        mark = len(list(temp))
        
        # if there is a rule exist, has the same with item_set, we consider this is dupulicate
        
        for k,v in temp.items():
            if k in item_set :
                if v == item_set[k]:
                    count += 1
                    if count == mark:
                        result = True
    return result

### 5 get final count and confidence measure

In [10]:
def get_confidence(antecedent_episode_list, antecedent_window, antecedent_data, 
                   consequent_episode_list, consequent_window, consequent_data, 
                   min_supp = 5, lag = 1, min_conf = 0.1):
    
    first_col = "episode/rule"
    second_col = "consequent"
    third_col = "Minimal occurrences"
    forth_col = "Support"
    fifth_col = "confidence"
    
    result_df = pd.DataFrame(columns = [first_col,second_col,third_col,forth_col, fifth_col])
    for i in range(len(consequent_episode_list)):
        for j in range(len(antecedent_episode_list)):
            result = count_association(antecedent_episode_list[j], antecedent_window, antecedent_data, 
                                      consequent_episode_list[i], consequent_window, consequent_data, 
                                      min_supp, lag, min_conf)
            if len(result) > 0:
                df2 = create_final_dataframe(result)
                
                result_df = pd.concat([result_df, df2])
    
    return result_df

def count_association(item1, win1, data1, item2, win2, data2, min_supp, lag, min_conf):
    
    ante_count = 0
    conse_count = 0
    
    
    ante_begin = 0
    ante_end = 0
    conse_begin = 0
    conse_end = 0
    
    rule = ""
    conse = ""
    ante_occur = ""
    conse_occur = ""
    occur_list = []
    conf = 0
    # we want to loop based on data row index
    for i in range(len(data1)):
                
        # we need to concern about the whole length may out of index, thus we need to check if index in dataframe range 
        # when processing it
                
        # first we need to check if all the epsiode in antecedent is exist
        # we will collect the count of exist, see if the number match with the number of elements
        # we will also collect the begin and end of each 
        # the begin and end of whole epsiode is the min-begin and max-end
        # in the search of consequent, we begin with the max-end of antecedent
        antecedent_count = 0
        
        ante_end_list = []    
        for k,v in item1.items():
            result, end_pos = check_episode_exist(k ,v, data1, i, win1)
            if result:
                antecedent_count += 1
                ante_end_list.append(end_pos)
            
        if antecedent_count == len(item1):
            ante_begin = i
            ante_end = max(ante_end_list)
            #print("pass")
            ante_count += 1
            ante_occur = "{}-{}".format(ante_begin, ante_end)
  
            
            # second we need to check when this item is exist, does consequent also exist
            # we need search from the position of antecedent max-end, with a distance of lag
            # if we deal with single dimension
                # if we deal with single item, then we find it - done
                # if we deal with mulitple item, we find the begining item inside distance of lag, and then find
                # right sequence inside window - done
            # if we deal with multiple dimension
                # we need to do every instance like above, and check the count
                # when the exist count is match the number of its dimension - done
                    
            # whatever happened here will not influence the count of antecedent, it always add 1

            for j in range(ante_end, ante_end + lag + 1):
                
                consequent_count = 0
                consequent_end_list = []
                for a,b in item2.items():
                    result, end_pos = check_episode_exist(a,b, data2, i, win2)
                    if result:
                        consequent_count += 1
                        consequent_end_list.append(end_pos)
                
                # to avoid repeat count, we only going to find once
                if consequent_count == len(item2):
                    conse_count += 1
                    conse_begin = j
                    conse_end = max(ante_end_list)
                    conse_occur = "{}-{}".format(conse_begin, conse_end)
                    occur_list.append([ante_occur, conse_occur])
                    break;
        
        
    if (ante_count > min_supp) & (conse_count > min_supp):
        if conse_count / ante_count > min_conf:
            
            for k,v in item1.items():
                rule += " {}".format(v)
                
            for k,v in item2.items():
                conse += " {}".format(v)
            
            rule += "lag{}".format(lag)
            conf = conse_count / ante_count     
            
            return [rule, conse, occur_list, conse_count, conf]
    
    
    return []

def check_episode_exist(space, episode, data, begin_index, window):
    
    local_data = data[space].copy()
    local_mark = len(episode)
    local_count = 0
    end_position = 0
    result = False
    for i in range(begin_index, begin_index + 1 + window):
        if i in local_data.index:
            #print(i, local_data.iloc[i] , local_mark, episode)
            if local_data.iloc[i] == episode[local_count][0]:
                local_count += 1
            if local_count == local_mark:
                end_position = i
                result = True
                break;
    
    return result, end_position

### 6. create final dataframe

In [11]:
def create_final_dataframe(result):
    
    first_col = "episode/rule"
    second_col = "consequent"
    third_col = "Minimal occurrences"
    forth_col = "Support"
    fifth_col = "confidence"
    
    first_list = []
    second_list = []
    third_list = []
    forth_list = []
    fifth_list = []
    
    df = pd.DataFrame(columns = [first_col,second_col,third_col,forth_col, fifth_col])
    
    if len(result) > 0:     
        result0 = result[0].replace("(", "")
        result0 = result0.replace(")", "")
        first_list.append(result0)
        result1 = result[1].replace("(", "")
        result1 = result1.replace(")", "")
        second_list.append(result1)
        third_list.append(result[2])
        forth_list.append(result[3])
        fifth_list.append(result[4])
    
        data = {first_col:  first_list,
                second_col: second_list,
                third_col: third_list,
                forth_col: forth_list,
                fifth_col: fifth_list
                }

        df = pd.DataFrame (data, columns = [first_col,second_col,third_col,forth_col,fifth_col])
        
    return df


# Complete packge of MOWCATL

In [12]:
def MOWCATL_algorithm(cleaned_data, target_data, min_supp = 15, window_antecedent = 3, window_consequent = 3, lag = 1, conf = 0.1):
    
    # init all k=1 epsiode
    antecedent_L1 = init_all_epsiode(cleaned_data, min_supp)
    consequent_L1 = init_all_epsiode(target_data, min_supp)
    
    # generate all epsiode space
    antecedent_item_sets, antecedent_item_sets_occurences, antecedent_item_sets_count = get_all_epsiodes(cleaned_data, antecedent_L1, min_supp, window_antecedent)
    consequent_item_sets, consequent_item_sets_occurences, consequent_item_sets_count = get_all_epsiodes(target_data, consequent_L1, min_supp, window_consequent)
    
    # visual initial epsiode space
    init_df = create_init_dataframe(antecedent_item_sets, antecedent_item_sets_occurences, antecedent_item_sets_count,
                         consequent_item_sets, consequent_item_sets_occurences, consequent_item_sets_count)
    
    # create the combination of epsiode space
    antecedent_space_name_order = cleaned_data.columns.values
    consequent_space_name_order = target_data.columns.values

    ante_Ln = get_space_epsiode_combination(antecedent_item_sets,antecedent_space_name_order)
    conse_Ln = get_space_epsiode_combination(consequent_item_sets,consequent_space_name_order)
    
    # final association rules
    result_df = get_confidence(ante_Ln, window_antecedent, cleaned_data, conse_Ln, window_consequent, target_data, min_supp, lag, conf)
    
    return init_df, result_df

In [13]:
df1, df2 = MOWCATL_algorithm(cleaned_data, target_data, min_supp = 5, window_antecedent = 3, window_consequent = 3, lag = 1, conf = 0.5)

this is  0 col
['no_cases'] 12 is support
this is  1 col
['no_deaths'] 14 is support
this is  2 col
['School_normal'] 14 is support
this is  3 col
['Workplace_normal'] 14 is support
this is  4 col
['Public_events_normal'] 14 is support
['public_events_cancel_Required'] 6 is support
this is  5 col
['No_gatherings_restrictions'] 14 is support
['Restrictions_ten_ppl_less'] 6 is support
this is  6 col
['transport_normal'] 16 is support
this is  7 col
['stay_home_not_require'] 16 is support
this is  8 col
['no_movement_restrictions'] 14 is support
['restrict_movement'] 7 is support
this is  9 col
['border_closure'] 7 is support
['no_travel_control'] 13 is support
this is  10 col
['cover_less_than_half_income'] 7 is support
['no_income_support'] 14 is support
this is  11 col
['broad_relief'] 7 is support
['no_relie'] 14 is support
this is  12 col
['coordinated_public_information_campaign'] 9 is support
['no_campaign'] 12 is support
this is  13 col
['no_testing_policy'] 12 is support
['open_p

{4: (['Public_events_normal'],), 13: (['no_testing_policy'],), 8: (['no_movement_restrictions'],)}
duplicate
{4: (['Public_events_normal'],), 13: (['no_testing_policy'],)}
{9: (['no_travel_control'],), 13: (['no_testing_policy'],)}
item allow to merge
{4: (['Public_events_normal'],), 13: (['no_testing_policy'],), 9: (['no_travel_control'],)}
duplicate
{4: (['Public_events_normal'],), 13: (['no_testing_policy'],)}
{10: (['no_income_support'],), 13: (['no_testing_policy'],)}
item allow to merge
{4: (['Public_events_normal'],), 13: (['no_testing_policy'],), 10: (['no_income_support'],)}
duplicate
{4: (['Public_events_normal'],), 13: (['no_testing_policy'],)}
{11: (['no_relie'],), 13: (['no_testing_policy'],)}
item allow to merge
{4: (['Public_events_normal'],), 13: (['no_testing_policy'],), 11: (['no_relie'],)}
duplicate
{4: (['Public_events_normal'],), 13: (['no_testing_policy'],)}
{12: (['coordinated_public_information_campaign'],), 13: (['no_testing_policy'],)}
item allow to merge
{4: 

duplicate
{4: (['Public_events_normal'],), 9: (['no_travel_control'],), 12: (['coordinated_public_information_campaign'],)}
{5: (['No_gatherings_restrictions'],), 9: (['no_travel_control'],), 12: (['coordinated_public_information_campaign'],)}
item allow to merge
{4: (['Public_events_normal'],), 9: (['no_travel_control'],), 12: (['coordinated_public_information_campaign'],), 5: (['No_gatherings_restrictions'],)}
duplicate
{4: (['Public_events_normal'],), 9: (['no_travel_control'],), 12: (['coordinated_public_information_campaign'],)}
{8: (['no_movement_restrictions'],), 9: (['no_travel_control'],), 12: (['coordinated_public_information_campaign'],)}
item allow to merge
{4: (['Public_events_normal'],), 9: (['no_travel_control'],), 12: (['coordinated_public_information_campaign'],), 8: (['no_movement_restrictions'],)}
duplicate
{4: (['Public_events_normal'],), 9: (['no_travel_control'],), 12: (['coordinated_public_information_campaign'],)}
{9: (['no_travel_control'],), 10: (['no_income_s

duplicate
{5: (['No_gatherings_restrictions'],), 13: (['no_testing_policy'],), 14: (['no_contact_tracing'],)}
{12: (['coordinated_public_information_campaign'],), 13: (['no_testing_policy'],), 14: (['no_contact_tracing'],)}
item allow to merge
{5: (['No_gatherings_restrictions'],), 13: (['no_testing_policy'],), 14: (['no_contact_tracing'],), 12: (['coordinated_public_information_campaign'],)}
duplicate
{5: (['No_gatherings_restrictions'],), 13: (['no_testing_policy'],), 14: (['no_contact_tracing'],)}
{12: (['no_campaign'],), 13: (['no_testing_policy'],), 14: (['no_contact_tracing'],)}
item allow to merge
{5: (['No_gatherings_restrictions'],), 13: (['no_testing_policy'],), 14: (['no_contact_tracing'],), 12: (['no_campaign'],)}
duplicate
{8: (['no_movement_restrictions'],), 9: (['no_travel_control'],), 10: (['no_income_support'],)}
{8: (['no_movement_restrictions'],), 9: (['no_travel_control'],), 11: (['no_relie'],)}
item allow to merge
{8: (['no_movement_restrictions'],), 9: (['no_trave

{4: (['Public_events_normal'],), 5: (['No_gatherings_restrictions'],), 8: (['no_movement_restrictions'],), 13: (['no_testing_policy'],), 9: (['no_travel_control'],)}
duplicate
{4: (['Public_events_normal'],), 5: (['No_gatherings_restrictions'],), 8: (['no_movement_restrictions'],), 13: (['no_testing_policy'],)}
{5: (['No_gatherings_restrictions'],), 8: (['no_movement_restrictions'],), 10: (['no_income_support'],), 13: (['no_testing_policy'],)}
item allow to merge
{4: (['Public_events_normal'],), 5: (['No_gatherings_restrictions'],), 8: (['no_movement_restrictions'],), 13: (['no_testing_policy'],), 10: (['no_income_support'],)}
duplicate
{4: (['Public_events_normal'],), 5: (['No_gatherings_restrictions'],), 8: (['no_movement_restrictions'],), 13: (['no_testing_policy'],)}
{5: (['No_gatherings_restrictions'],), 8: (['no_movement_restrictions'],), 11: (['no_relie'],), 13: (['no_testing_policy'],)}
item allow to merge
{4: (['Public_events_normal'],), 5: (['No_gatherings_restrictions'],), 8

...

{4: (['Public_events_normal'],), 8: (['no_movement_restrictions'],), 11: (['no_relie'],), 12: (['no_campaign'],), 13: (['no_testing_policy'],)}
{4: (['Public_events_normal'],), 8: (['no_movement_restrictions'],), 11: (['no_relie'],), 12: (['no_campaign'],), 14: (['no_contact_tracing'],)}
item allow to merge
{4: (['Public_events_normal'],), 8: (['no_movement_restrictions'],), 11: (['no_relie'],), 12: (['no_campaign'],), 13: (['no_testing_policy'],), 14: (['no_contact_tracing'],)}
{4: (['Public_events_normal'],), 8: (['no_movement_restrictions'],), 11: (['no_relie'],), 12: (['no_campaign'],), 13: (['no_testing_policy'],)}
{4: (['Public_events_normal'],), 8: (['no_movement_restrictions'],), 11: (['no_relie'],), 13: (['no_testing_policy'],), 14: (['no_contact_tracing'],)}
item allow to merge
{4: (['Public_events_normal'],), 8: (['no_movement_restrictions'],), 11: (['no_relie'],), 12: (['no_campaign'],), 13: (['no_testing_policy'],), 14: (['no_contact_tracing'],)}
duplicate
{4: (['Public_ev

{5: (['No_gatherings_restrictions'],), 8: (['no_movement_restrictions'],), 9: (['no_travel_control'],), 11: (['no_relie'],), 13: (['no_testing_policy'],)}
{5: (['No_gatherings_restrictions'],), 8: (['no_movement_restrictions'],), 9: (['no_travel_control'],), 12: (['coordinated_public_information_campaign'],), 13: (['no_testing_policy'],)}
item allow to merge
{5: (['No_gatherings_restrictions'],), 8: (['no_movement_restrictions'],), 9: (['no_travel_control'],), 11: (['no_relie'],), 13: (['no_testing_policy'],), 12: (['coordinated_public_information_campaign'],)}
duplicate
{5: (['No_gatherings_restrictions'],), 8: (['no_movement_restrictions'],), 9: (['no_travel_control'],), 11: (['no_relie'],), 13: (['no_testing_policy'],)}
{5: (['No_gatherings_restrictions'],), 8: (['no_movement_restrictions'],), 9: (['no_travel_control'],), 12: (['no_campaign'],), 13: (['no_testing_policy'],)}
item allow to merge
{5: (['No_gatherings_restrictions'],), 8: (['no_movement_restrictions'],), 9: (['no_trave

{9: (['no_travel_control'],), 11: (['no_relie'],), 12: (['no_campaign'],), 13: (['no_testing_policy'],), 14: (['no_contact_tracing'],)}
item allow to merge
{5: (['No_gatherings_restrictions'],), 9: (['no_travel_control'],), 12: (['no_campaign'],), 13: (['no_testing_policy'],), 14: (['no_contact_tracing'],), 11: (['no_relie'],)}
duplicate
{5: (['No_gatherings_restrictions'],), 10: (['no_income_support'],), 11: (['no_relie'],), 12: (['coordinated_public_information_campaign'],), 13: (['no_testing_policy'],)}
{5: (['No_gatherings_restrictions'],), 10: (['no_income_support'],), 11: (['no_relie'],), 12: (['coordinated_public_information_campaign'],), 14: (['no_contact_tracing'],)}
item allow to merge
{5: (['No_gatherings_restrictions'],), 10: (['no_income_support'],), 11: (['no_relie'],), 12: (['coordinated_public_information_campaign'],), 13: (['no_testing_policy'],), 14: (['no_contact_tracing'],)}
{5: (['No_gatherings_restrictions'],), 10: (['no_income_support'],), 11: (['no_relie'],), 12

{4: (['Public_events_normal'],), 5: (['No_gatherings_restrictions'],), 8: (['no_movement_restrictions'],), 9: (['no_travel_control'],), 12: (['coordinated_public_information_campaign'],), 13: (['no_testing_policy'],), 11: (['no_relie'],)}
duplicate
{4: (['Public_events_normal'],), 5: (['No_gatherings_restrictions'],), 8: (['no_movement_restrictions'],), 9: (['no_travel_control'],), 12: (['coordinated_public_information_campaign'],), 13: (['no_testing_policy'],)}
{4: (['Public_events_normal'],), 5: (['No_gatherings_restrictions'],), 8: (['no_movement_restrictions'],), 12: (['coordinated_public_information_campaign'],), 13: (['no_testing_policy'],), 14: (['no_contact_tracing'],)}
item allow to merge
{4: (['Public_events_normal'],), 5: (['No_gatherings_restrictions'],), 8: (['no_movement_restrictions'],), 9: (['no_travel_control'],), 12: (['coordinated_public_information_campaign'],), 13: (['no_testing_policy'],), 14: (['no_contact_tracing'],)}
duplicate
{4: (['Public_events_normal'],), 5

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [14]:
df1

,episode/rule,consequent,Minimal occurrences,Support,confidence
0,"([Public_events_normal],)",#,"[1-1, 2-2, 3-3, 4-4, 5-5, 6-6, 7-7, 8-8, 9-9, ...",14,#
1,"([No_gatherings_restrictions],)",#,"[1-1, 2-2, 3-3, 4-4, 5-5, 6-6, 7-7, 8-8, 9-9, ...",14,#
2,"([no_movement_restrictions],)",#,"[1-1, 2-2, 3-3, 4-4, 5-5, 6-6, 7-7, 8-8, 9-9, ...",14,#
3,"([no_travel_control],)",#,"[1-1, 2-2, 3-3, 4-4, 5-5, 6-6, 7-7, 8-8, 9-9, ...",13,#
4,"([no_income_support],)",#,"[1-1, 2-2, 3-3, 4-4, 5-5, 6-6, 7-7, 8-8, 9-9, ...",14,#
5,"([no_relie],)",#,"[1-1, 2-2, 3-3, 4-4, 5-5, 6-6, 7-7, 8-8, 9-9, ...",14,#
6,"([coordinated_public_information_campaign],)",#,"[13-13, 14-14, 15-15, 16-16, 17-17, 18-18]",6,#
7,"([no_campaign],)",#,"[1-1, 2-2, 3-3, 4-4, 5-5, 6-6, 7-7, 8-8, 9-9, ...",12,#
8,"([no_testing_policy],)",#,"[1-1, 2-2, 3-3, 4-4, 5-5, 6-6, 7-7, 8-8, 9-9, ...",12,#
9,"([no_contact_tracing],)",#,"[1-1, 2-2, 3-3, 4-4, 5-5, 6-6, 7-7, 8-8, 9-9, ...",12,#


In [15]:
df2

,episode/rule,consequent,Minimal occurrences,Support,confidence
0,"['Public_events_normal'],lag1","['Full_time_employed_high'],","[[0-1, 1-1], [1-1, 1-1], [2-2, 2-2], [4-4, 4-4...",14,0.933333
0,"['No_gatherings_restrictions'],lag1","['Full_time_employed_high'],","[[0-1, 1-1], [1-1, 1-1], [2-2, 2-2], [4-4, 4-4...",14,0.933333
0,"['no_movement_restrictions'],lag1","['Full_time_employed_high'],","[[0-1, 1-1], [1-1, 1-1], [2-2, 2-2], [4-4, 4-4...",14,0.933333
0,"['no_travel_control'],lag1","['Full_time_employed_high'],","[[0-1, 1-1], [1-1, 1-1], [2-2, 2-2], [4-4, 4-4...",13,0.928571
0,"['no_income_support'],lag1","['Full_time_employed_high'],","[[0-1, 1-1], [1-1, 1-1], [2-2, 2-2], [4-4, 4-4...",14,0.933333
...,...,...,...,...,...
0,"['Public_events_normal'], ['No_gatherings_res...","['Full_time_employed_medium'], ['Part_time_em...","[[2-2, 2-2], [3-3, 3-3], [4-4, 4-4], [5-5, 5-5...",9,0.692308
0,"['Public_events_normal'], ['No_gatherings_res...","['Full_time_employed_medium'], ['Part_time_em...","[[2-2, 2-2], [3-3, 3-3], [4-4, 4-4], [5-5, 5-5...",9,0.692308
0,"['Public_events_normal'], ['no_movement_restr...","['Full_time_employed_medium'], ['Part_time_em...","[[2-2, 2-2], [3-3, 3-3], [4-4, 4-4], [5-5, 5-5...",9,0.692308
0,"['No_gatherings_restrictions'], ['no_movement...","['Full_time_employed_medium'], ['Part_time_em...","[[2-2, 2-2], [3-3, 3-3], [4-4, 4-4], [5-5, 5-5...",9,0.692308


In [16]:
df2.to_csv(r"D:\Download\COVID19\MOWCATL result.csv", index = False, header=True)